In [ ]:
#!pip install transformers datasets

In [ ]:
#!pip install torch

In [ ]:
from transformers import pipeline

pipe = pipeline("question-answering", model="dbmdz/bert-base-turkish-128k-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers import DefaultDataCollator
from torchsummary import summary

#tokenizer = AutoTokenizer.from_pretrained("husnu/bert-base-turkish-128k-cased-finetuned_lr-2e-05_epochs-3")
#model = AutoModelForQuestionAnswering.from_pretrained("husnu/bert-base-turkish-128k-cased-finetuned_lr-2e-05_epochs-3")
#tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-cased")
#model = AutoModelForQuestionAnswering.from_pretrained("dbmdz/bert-base-turkish-128k-cased")

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-cased")
model = AutoModelForQuestionAnswering.from_pretrained("dbmdz/bert-base-turkish-128k-cased")

print(model)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [ ]:
from google.colab import drive
import json

drive.mount('/content/drive')
with open(r"/content/drive/MyDrive/Wiki_Dataset_Final.json", "r") as read_file:
  data_dict = json.load(read_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
contexts = []
questions = []
answers = []

for entry in data_dict['data']:
  text = entry['text']
  input_ids = tokenizer.encode(text, add_special_tokens=True)

  if (len(input_ids) <= 512):

    for qa in entry['qas']:
      question = qa['question']
      answer = qa['answer']
      answer_start = qa['answer']
      answer_start = qa['answer_start']
      answer_end = qa['answer_end']

      answer_start_end_dict = { }
      answer_start_end_dict['answer'] = answer
      answer_start_end_dict['answer_start'] = answer_start
      answer_start_end_dict['answer_end'] = answer_end

      contexts.append(text)
      questions.append(question)
      answers.append(answer_start_end_dict)



Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors


In [ ]:

max_len = 0

for text in contexts:

    input_ids = tokenizer.encode(text, add_special_tokens=True)

    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)


Max sentence length:  502


In [ ]:

train_contexts = contexts[: int(len(contexts) * 0.95)]
val_contexts = contexts[int(len(contexts) * 0.95) :]
train_answers = answers[: int(len(answers) * 0.95)]
val_answers = answers[int(len(answers) * 0.95) :]
train_questions = questions[: int(len(questions) * 0.95)]
val_questions = questions[int(len(contexts) * 0.95) :]
"""
train_contexts = contexts
train_answers = answers
train_questions = questions
"""
print(len(train_questions))
print(len(val_questions))

4707
248


In [ ]:
train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

for key, value in train_encodings.items() :
    print (key)

input_ids
token_type_ids
attention_mask


In [ ]:
"""
print("val answers:")
for i in range(50):
  print(train_answers[i])

print("answers len:")
print(len(answers))
#print(len(train_encodings.items()))
"""

def add_token_positions(encodings, answers):
  answer_start_positions = []
  answer_end_positions = []

  for i in range(len(answers)):
    answer_start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
    answer_end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

    if answer_start_positions[-1] is None:
      answer_start_positions[-1] = tokenizer.model_max_length
    if answer_end_positions[-1] is None:
      answer_end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': answer_start_positions, 'end_positions': answer_end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

for key, value in train_encodings.items() :
    print (key)

input_ids
token_type_ids
attention_mask
start_positions
end_positions


In [ ]:
import torch

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      return item

    def __len__(self):
        return len(self.encodings.input_ids)


train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)
#print(train_dataset[0])
print(len(train_dataset))
print(len(train_encodings))


4707
5


'from transformers import TrainingArguments, Trainer\n\ntrain_args = TrainingArguments(\n    output_dir="bert-finetune-bitirme-16batch",\n    evaluation_strategy="epoch",\n    learning_rate=2e-5,\n    per_device_train_batch_size=16,\n    per_device_eval_batch_size=16,\n    num_train_epochs=3,\n    weight_decay=0.01,\n)\n\ntrainer = Trainer(\n    model=model,\n    args=train_args,\n    train_dataset=train_dataset,\n    eval_dataset=val_dataset,\n\n)'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from torch.utils.data import DataLoader
from transformers import AdamW
import pdb
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
print(len(train_loader))
print(len(train_dataset))

optim = AdamW(model.parameters(), lr=2e-5)

#print(len(train_loader))

for epoch in range(2):
    print("Epoch {}".format(epoch))
    batch_num = 0
    for batch in train_loader:
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        #outputs = model.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        #outputs = model.bert(input_ids, attention_mask=attention_mask, answer_start=start_positions, answer_end=end_positions)
        loss = outputs[0]
        #loss.mean().backward()
        loss.backward()
        optim.step()
        if batch_num % 10 == 0:
          print("Batch {}: Loss = {}".format(batch_num, loss.item()))
        batch_num += 1

model.eval()


Using device: cuda

Tesla T4
Memory Usage:
Allocated: 2.8 GB
Cached:    12.8 GB
295
4707
Epoch 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Batch 0: Loss = 1.5224076509475708
Batch 10: Loss = 1.0995242595672607
Batch 20: Loss = 0.850131094455719
Batch 30: Loss = 1.2519258260726929
Batch 40: Loss = 0.8630661368370056
Batch 50: Loss = 1.0343854427337646
Batch 60: Loss = 1.3816474676132202
Batch 70: Loss = 1.0806645154953003
Batch 80: Loss = 0.3219192326068878
Batch 90: Loss = 1.9127004146575928
Batch 100: Loss = 1.104391098022461
Batch 110: Loss = 1.191523551940918
Batch 120: Loss = 0.9998846054077148
Batch 130: Loss = 1.3146454095840454
Batch 140: Loss = 1.023534893989563
Batch 150: Loss = 0.8943449258804321
Batch 160: Loss = 1.6401782035827637
Batch 170: Loss = 0.9291471242904663
Batch 180: Loss = 0.9780365228652954
Batch 190: Loss = 0.26940563321113586
Batch 200: Loss = 1.210416316986084
Batch 210: Loss = 1.1520577669143677
Batch 220: Loss = 0.23170311748981476
Batch 230: Loss = 0.20479337871074677
Batch 240: Loss = 0.2758384346961975
Batch 250: Loss = 0.4890936315059662
Batch 260: Loss = 0.5077285766601562
Batch 270: Los

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids,
                            attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            #loss, start_scores, end_scores = outputs
            loss = outputs[0]
            start_scores = outputs[1]
            end_scores = outputs[2]

            predicted_start = torch.argmax(start_scores, dim=1)
            predicted_end = torch.argmax(end_scores, dim=1)

            correct_predictions += ((predicted_start == start_positions) & (predicted_end == end_positions)).sum().item()
            total_samples += start_positions.size(0)

    accuracy = correct_predictions / total_samples
    return accuracy

model.to(device)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

accuracy = evaluate_model(model, val_loader, device)
print("Test Accuracy: {:.2%}".format(accuracy))

Test Accuracy: 9.68%


In [ ]:
from sklearn.metrics import f1_score

val_loader = DataLoader(train_dataset, batch_size=16)

def evaluate_model_f1(model, dataloader, device):
    model.eval()
    all_true_start = []
    all_pred_start = []
    all_true_end = []
    all_pred_end = []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)

            outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            loss = outputs[0]
            start_scores = outputs[1]
            end_scores = outputs[2]

            predicted_start = torch.argmax(start_scores, dim=1)
            predicted_end = torch.argmax(end_scores, dim=1)

            all_true_start.extend(start_positions.cpu().numpy())
            all_pred_start.extend(predicted_start.cpu().numpy())
            all_true_end.extend(end_positions.cpu().numpy())
            all_pred_end.extend(predicted_end.cpu().numpy())

    f1_start = f1_score(all_true_start, all_pred_start, average='micro')
    f1_end = f1_score(all_true_end, all_pred_end, average='micro')
    f1_score_total = (f1_start + f1_end) / 2.0

    return f1_score_total

model.to(device)
f1_score_result = evaluate_model_f1(model, val_loader, device)
print("Test F1 Score: {:.2%}".format(f1_score_result))

Test F1 Score: 56.22%


In [ ]:
"""from huggingface_hub import notebook_login

notebook_login()

model.push_to_hub("ykakca/bert-bitirme-128k")
tokenizer.push_to_hub("ykakca/bert-bitirme-128k")"""

In [ ]:

"""
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("ykakca/bert-ytu-bitirme-fine-tuned")
model = AutoModelForQuestionAnswering.from_pretrained("ykakca/bert-ytu-bitirme-fine-tuned")
model = model.to(device)
"""